<a href="https://www.kaggle.com/code/ishansrivastava1308/commit-notebook-of-spotify-data-scraper?scriptVersionId=186860228" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install pytube --upgrade
!pip install spotipy
!pip install ytmusicapi
# !pip install yt-dlp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.5 MB/s eta 0:00:00


In [2]:
import warnings

# Suppress the specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.")

In [3]:
from ytmusicapi import YTMusic
import ytmusicapi
from pprint import pprint
import os
from pytube import YouTube
import urllib.request
import requests
import string
from spotipy.client import Spotify
from spotipy import SpotifyOAuth
from spotipy.oauth2 import SpotifyOauthError, SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import concurrent.futures
import time
import threading
from pydub import AudioSegment
from IPython.display import Audio
from dotenv import load_dotenv
import multiprocessing
from tqdm import tqdm
from IPython.display import Audio
from tensorflow.keras.utils import Progbar
# import yt_dlp

2024-07-04 16:50:01.007777: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 16:50:01.007945: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 16:50:01.189547: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import pytube.innertube as pti

In [5]:
pti._token_file = '/kaggle/working/tokens.json'
pti._cache_dir = '/kaggle/working'

# Making Kaggle Dataset

In [6]:
import os
import json

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")

In [7]:
meta = dict(
    id="ishansrivastava1308/spotify-dataset",
    title="Spotify Dataset",
    isPrivate=True,
    licenses=[dict(name="other")]
)
os.makedirs('/kaggle/working/dataset', exist_ok = True)
with open(os.path.join('dataset','dataset-metadata.json'), 'w') as f:
    json.dump(meta, f)

In [8]:
# !kaggle datasets create -p /kaggle/working/dataset --dir-mode tar

In [9]:
# !kaggle datasets version -m"Added more songs and data" -p /kaggle/working/dataset --dir-mode tar

# Scrapper Class Definition

In [10]:
df_columns = ['track_name', 'track_id', 'track_number', 'disc_number', 'duration_ms',
       'explicit', 'popularity', 'preview_url', 'isrc', 'album_name',
       'album_id', 'album_type', 'album_total_tracks', 'album_release_date',
       'album_release_date_precision', 'album_images', 'popular_artist',
       'popular_artist_id', 'artist_names', 'artist_ids', 'combined_genres',
       'artist_popularity', 'artist_followers', 'external_url', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence',
       'lyrics', 'audio_file_path', 'video_id']

In [11]:
class YTScraper:
    def __init__(
        self,
        token_file_path="oauth.json",
    ):
        self.yt_music = YTMusic(token_file_path)
        YouTube(
            "https://www.youtube.com/watch?v=dQw4w9WgXcQ",
            use_oauth=True,
            allow_oauth_cache=True,
        ).streams.filter(only_audio=True).first().download()

    def get_search_results(self, search_str):
        return self.yt_music.search(search_str, filter="songs")
    
    def get_best_video_id(self,search_str):
        search_result = self.get_search_results(search_str=search_str)
        video_id = None
        try:
            for item in search_result:
                if item["resultType"] in ["song"] and item["category"] == "Songs":
                    video_id = item["videoId"]
                    break
        except Exception as e:
            print(f"Exception in get_best_video_id message : {e}")
        return video_id
    
    def get_song_from_video_id(self,video_id,output_path = './', audio_format = 'mp3', target_sr = 16000, num_channels = 1):
        os.makedirs(output_path, exist_ok=True)
        temp_file = os.path.join(output_path, f"{video_id}.mp4")
        download_path = os.path.join(output_path,f"{video_id}.{audio_format}")
        try:
            yt = YouTube(f"https://youtube.com/watch?v={video_id}")
            yt.streams.filter(only_audio=True).first().download(
                output_path=output_path, filename=f"{video_id}.mp4"
            )
            if os.path.exists(temp_file):
                #         print(f"Downloaded successfully: {temp_file}")
                audio = AudioSegment.from_file(temp_file, format="mp4")
                audio = audio.set_frame_rate(target_sr)
                audio = audio.set_channels(num_channels)
                wav_path = temp_file[:-4] + audio_format
                audio.export(wav_path, format=audio_format)
                os.remove(temp_file)
            return download_path
        except Exception as e:
            return None
            
    def get_lyrics_video_id(self, video_id):
        video = self.yt_music.get_watch_playlist(
            videoId=video_id,
        )
        lyrics_id = video["lyrics"]
        lyrics = None
        if lyrics_id:
            try:
                lyrics = self.yt_music.get_lyrics(lyrics_id)
            except:
                pass
        return lyrics
    
#     def threaded_download_song_by_video_id_list(
#         self, video_id_list, output_path="songs", max_workers=50
#     ):
#         data = []
#         with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#             # List of tasks to be executed by the thread pool
#             tasks = []
#                         output_path_args = [output_path] * len(video_id_list)

#             for args in zip(video_id_list, output_path_args):
#                 fut = executor.submit(self.download_song_by_video_id, *args)

#             tasks = [
                
#             ]
#             success = 0
#             fails = 0

#             with tqdm(total=len(video_id_list)) as pbar:
#                 for future in concurrent.futures.as_completed(tasks):
#                     try:
#                         data.append(future.result())
#                         pbar.update(1)
#                         success += 1
#                     except Exception as exc:
#                         print(f"Task generated an exception: {exc}")
#                         fails += 1
#             print({"success": success, "fail": fails})
#         return data

    def scrap_by_search(
        self,
        search_str,
        download_params={
            "include_audio": True,
            "include_lyrics": True,
            "output_path": "songs",
        },
    ):
        include_lyrics = download_params["include_lyrics"]
        include_audio = download_params["include_audio"]
        output_path = download_params["output_path"]

        search_result = self.get_search_results(search_str=search_str)
        video_id = None
        try:
            for item in search_result:
                if item["resultType"] in ["song"] and item["category"] == "Songs":
                    video_id = item["videoId"]
                    break
        except Exception as e:
            print(e)
            pass
        lyrics = {"lyrics": None, "source": None}
        download_path = None
        if video_id:
            if include_lyrics:
                lyrics = self.get_lyrics(video_id=video_id)
                if not lyrics:
                    lyrics = {"lyrics": None, "source": None}
            if include_audio:
                download_path = self.download_song_by_video_id(
                    video_id=video_id, output_path=output_path
                )

        return {
            "lyrics": lyrics["lyrics"],
            "audio_file_path": download_path,
            "video_id": video_id,
        }

In [12]:
class SpotifyScraper_v1:
    def __init__(
        self,
        client_id,
        client_secret,
#         redirect_uri,
        scope = "user-library-read playlist-read-private playlist-read-collaborative"
        
    ):
        self.df = pd.DataFrame(columns = df_columns)
        self.yt_scraper = YTScraper()
        self._credentials = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
        self.sp = Spotify(auth = self.get_access_token())
        self.download_params = {
            'include_audio' : True,
            'include_lyrics' : True,
            'output_path' : 'songs'
        }
            
    def get_access_token(self):
        # Get access token
        access_token = self._credentials.get_access_token(as_dict = False)
        if not access_token:
            raise Exception("Access Token Not Found")
        return access_token

    def get_auth_header(self):
        return {"Authorization": "Bearer " + self.get_access_token()}
    # Function to fetch user playlists
    
    def get_user_playlists(self):
        print("Retrieving user playlists...")
        headers = self.get_auth_header()
        response = requests.get("https://api.spotify.com/v1/me/playlists", headers=headers)
        response_json = response.json()
        # for item in response_json["items"]:
        #     playlists[item["name"]] = item["id"]
        print("Playlists retrieved successfully.")
        return response_json
    
    @staticmethod
    def sanitize_filename(filename):
        valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
        return ''.join(c for c in filename if c in valid_chars)

    def get_track_info_by_id(self, track_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/tracks/{track_id}", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_playlist_info_by_id(self, playlist_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_several_track_info_by_id(self, track_ids_str):
        for i in range(3):
            headers = self.get_auth_header()
            response = requests.get(f"https://api.spotify.com/v1/tracks?ids={track_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
                if (retry_after):
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(int(retry_after) + 1)
                else:
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(10)
            else:
                response_json = response.json()
                return response_json
    
    def get_several_artist_info_by_id(self, artist_ids_str):
        for i in range(3):
            headers = self.get_auth_header()
            response = requests.get(f"https://api.spotify.com/v1/artists?ids={artist_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
                if (retry_after):
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(int(retry_after) + 1)
                else:
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(10)
            else:
                response_json = response.json()
                return response_json
    
    def get_several_audio_feature_by_id(self, track_ids_str):
        for i in range(3):
            headers = self.get_auth_header()
            response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={track_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
                if (retry_after):
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(int(retry_after) + 1)
                else:
                    print(f"Retrying after {retry_after} seconds")
                    time.sleep(10)
            else:
                response_json = response.json()
                return response_json
    
    def destructure_artist_data(self,artists_list):
        artist_infos = []
        artist_genres = []
        artist_popularities = []
        artist_followers = []
        for artist_obj in artists_list:
            
            artist_infos.append({
                "name": artist_obj.get("name"),
                "id": artist_obj.get("id"),
                "genres": artist_obj.get("genres", []),
                "popularity": artist_obj.get("popularity"),
                "followers": artist_obj.get("followers", {}).get("total")
            })
            artist_genres.extend(artist_obj.get("genres", []))
            artist_popularities.append(artist_obj.get("popularity"))
            artist_followers.append(artist_obj.get("followers", {}).get("total"))

        result = {
            'artist_infos': artist_infos,
            'artist_genres': artist_genres,
            'artist_popularities': artist_popularities,
            'artist_followers': artist_followers,
        }
        return result

        
    def get_playlist_by_id(self,playlist_id : str):
        return self.sp.playlist_tracks(playlist_id = playlist_id)    
    
    def construct_track_info_dict(self,track_json = None, audio_features_json = None, artist_json = None):         
        if track_json is None:
            track_json = {}
        if audio_features_json is None:
            audio_features_json = {}
        
        most_popular_artist = {}
        if len(artist_json['artist_infos']):
            most_popular_artist = max(artist_json['artist_infos'], key=lambda x: x.get("popularity", 0))
        track_name = track_json.get('name', None)
        most_popular_artist_name = most_popular_artist.get('name', '')
        
        
        lyrics_audio_data = {
            "lyrics": None,
            "audio_file_path": None,
            "video_id": None,
        }
        
        if track_name:
            search_str = f"{track_name} {most_popular_artist_name}"
            lyrics_audio_data = self.yt_scraper.scrap_by_search(search_str=search_str, download_params = self.download_params)

        
        track_info = {
            "track_name": track_json.get("name"),
            "track_id": track_json.get("id"),
            "track_number": track_json.get("track_number"),
            "disc_number": track_json.get("disc_number"),
            "duration_ms": track_json.get("duration_ms"),
            "explicit": track_json.get("explicit"),
            "popularity": track_json.get("popularity"),
            "preview_url": track_json.get("preview_url"),
            "isrc": track_json.get("external_ids", {}).get("isrc"),
            "album_name": track_json.get("album", {}).get("name"),
            "album_id": track_json.get("album", {}).get("id"),
            "album_type": track_json.get("album", {}).get("album_type"),
            "album_total_tracks": track_json.get("album", {}).get("total_tracks"),
            "album_release_date": track_json.get("album", {}).get("release_date"),
            "album_release_date_precision": track_json.get("album", {}).get("release_date_precision"),
            "album_images": track_json.get("album", {}).get("images"),
            "popular_artist": most_popular_artist.get("name"),
            "popular_artist_id": most_popular_artist.get("id"),
            "artist_names": [artist.get("name") for artist in track_json.get("artists", [])],
            "artist_ids": [artist.get("id") for artist in track_json.get("artists", [])],
            "combined_genres": list(set(artist_json.get('artist_genres', []))),
            "artist_popularity": most_popular_artist.get("popularity"),
            "artist_followers": most_popular_artist.get("followers"),
            "external_url": track_json.get("external_urls", {}).get("spotify"),
            "acousticness": audio_features_json.get("acousticness"),
            "danceability": audio_features_json.get("danceability"),
            "energy": audio_features_json.get("energy"),
            "instrumentalness": audio_features_json.get("instrumentalness"),
            "key": audio_features_json.get("key"),
            "liveness": audio_features_json.get("liveness"),
            "loudness": audio_features_json.get("loudness"),
            "mode": audio_features_json.get("mode"),
            "speechiness": audio_features_json.get("speechiness"),
            "tempo": audio_features_json.get("tempo"),
            "time_signature": audio_features_json.get("time_signature"),
            "valence": audio_features_json.get("valence"),
        }
        track_info.update(lyrics_audio_data)
        return track_info
        
    def _get_single_threaded_data(self,track_obj, audio_feat_obj):
        artist_ids_list = [artist['id'] for artist in track_obj["artists"]]
        artists_id_str = ','.join(artist_ids_list[:50])
        artists_data = self.get_several_artist_info_by_id(artists_id_str)["artists"]
        destructured_artist_data = self.destructure_artist_data(artists_list=artists_data)  
        return self.construct_track_info_dict(track_obj, audio_feat_obj, destructured_artist_data)
    
    def _get_multi_threaded_data(self,tracks_json,audio_features_json, max_workers):
        data = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # List of tasks to be executed by the thread pool
            tasks = [
                executor.submit(self._get_single_threaded_data , *args) for args in zip(tracks_json, audio_features_json)
            ]
            success = 0
            fails = 0
            
            with tqdm(total=len(tracks_json)) as pbar:
                for future in concurrent.futures.as_completed(tasks):
                    try:
                        data.append(future.result())
                        pbar.update(1)
                        success+=1
                    except Exception as exc:
                        print(f'Task generated an exception: {exc}') 
                        fails+=1
            print({'success' : success, 'fail' : fails})
                
        return data
    
    def _get_multiprocessed_data(self,tracks_json,audio_features_json, max_workers):
        data = []
        with multiprocessing.Pool(processes=max_workers) as pool:
            # List of tasks to be executed by the thread pool
            tasks = [
                pool.apply_async(self._get_single_threaded_data , *args) for args in zip(tracks_json, audio_features_json)
            ]
            
            for task in tasks:
                # try:
                    result = task.get()
                    print(result)
                    data.append(task.get())
                # except Exception as exc:
                    # print(f'Task generated an exception: {exc}') 
        return data
    
    
    def get_several_track_data(self, track_ids_list : list[str], parallelization = 0, max_workers = 50, include_audio = True, include_lyrics = True, output_path = "songs"):
        data = []
        track_ids_list = list(set(track_ids_list))
        self.download_params['include_audio'] = include_audio
        self.download_params['include_lyrics'] = include_lyrics
        self.download_params['output_path'] = output_path
        with tqdm(total = len(track_ids_list)) as pbar:
            for idx in range(0,len(track_ids_list),50):
                track_id_str = ",".join(track_ids_list[idx:idx+50])
                tracks_json = self.get_several_track_info_by_id(track_id_str)["tracks"]
                audio_features_json = self.get_several_audio_feature_by_id(track_ids_str=track_id_str)["audio_features"]
                if parallelization == 1:
                    data.extend(self._get_multi_threaded_data(tracks_json,audio_features_json,max_workers))
                elif False and parallelization == 2:
                    data.extend(self._get_multiprocessed_data(tracks_json,audio_features_json,max_workers))
                else:
                    temp_data = []
                    for track_obj, audio_feat_obj in zip(tracks_json, audio_features_json):
                        
                        temp_data.append(self._get_single_threaded_data(track_obj, audio_feat_obj))
                        pbar.update(1)
                    data.extend(temp_data)
                    self.df = pd.concat([self.df,pd.DataFrame(temp_data)],ignore_index = True)
        return data
    
    def get_track_ids_from_playlist(self,*args,**kwargs):
        playlist_data = self.sp.playlist_tracks(*args,**kwargs)
        return [item['track']['id'] for item in playlist_data['items']]
    
    def get_track_ids_from_album(self,*args,**kwargs):
        album_data = self.sp.album_tracks(*args,**kwargs)
        return [item['id'] for item in album_data['items']]
    
    
      

In [13]:
class SpotifyScraper:
    def __init__(
        self,
        client_id,
        client_secret,
#         redirect_uri,
        scope = "user-library-read playlist-read-private playlist-read-collaborative",
        delay = 0.5,
        track_audio_feat_df = pd.DataFrame(),
        artists_df = pd.DataFrame(),   
        combined_df = pd.DataFrame(),
        lyrics_audio_df = pd.DataFrame(),
        track_video_df = pd.DataFrame(),
        
    ):
        self.delay = delay
        self.track_audio_feat_df = track_audio_feat_df
        self.artists_df = artists_df
        self.combined_df = combined_df
        self.lyrics_audio_df = lyrics_audio_df
        self.track_video_df = track_video_df
        
        self.yt_scraper = YTScraper()
        self._credentials = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret)
        self.sp = Spotify(auth = self.get_access_token())
        self.download_params = {
            'include_audio' : True,
            'include_lyrics' : True,
            'output_path' : 'songs'
        }
            
    def get_access_token(self):
        # Get access token
        access_token = self._credentials.get_access_token(as_dict = False)
        if not access_token:
            raise Exception("Access Token Not Found")
        return access_token

    def get_auth_header(self):
        return {"Authorization": "Bearer " + self.get_access_token()}
    # Function to fetch user playlists
    
    def get_user_playlists(self):
        print("Retrieving user playlists...")
        headers = self.get_auth_header()
        response = requests.get("https://api.spotify.com/v1/me/playlists", headers=headers)
        response_json = response.json()
        # for item in response_json["items"]:
        #     playlists[item["name"]] = item["id"]
        print("Playlists retrieved successfully.")
        return response_json
    
    @staticmethod
    def sanitize_filename(filename):
        valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
        return ''.join(c for c in filename if c in valid_chars)

    def get_track_info_by_id(self, track_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/tracks/{track_id}", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_playlist_info_by_id(self, playlist_id):
        headers = self.get_auth_header()
        response = requests.get(f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks", headers=headers)
        if not response.ok:
            raise Exception(f"{response.status_code} : {response.text}")
        response_json = response.json()
        return response_json
    
    def get_several_track_info_by_id(self, track_ids_str, delay = 0):
        for i in range(3):
            headers = self.get_auth_header()
            time.sleep(self.delay)
            response = requests.get(f"https://api.spotify.com/v1/tracks?ids={track_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
#                 if (retry_after):
#                     print(f"Retrying after {retry_after} seconds")
#                     time.sleep(int(retry_after) + 1)
#                 else:
#                     print(f"Retrying after {retry_after} seconds")
#                     time.sleep(10)
                raise Exception(f'Rate limit Hit for track_info retry after : {retry_after}')
            else:
                response_json = response.json()
                return response_json
    
    def get_several_artist_info_by_id(self, artist_ids_str, delay = 0):
        for i in range(3):
            headers = self.get_auth_header()
            time.sleep(self.delay)
            response = requests.get(f"https://api.spotify.com/v1/artists?ids={artist_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
#                 if (retry_after):
#                     print(f"Retrying after {retry_after} seconds")
#                     time.sleep(int(retry_after) + 1)
#                 else:
#                     print(f"Retrying after {retry_after} seconds")
#                     time.sleep(10)
                raise Exception(f'Rate limit Hit for artist_info retry after : {retry_after}')

                  
            else:
                response_json = response.json()
                return response_json
    
    def get_several_audio_feature_by_id(self, track_ids_str, delay = 0):
        for i in range(3):
            headers = self.get_auth_header()
            time.sleep(self.delay)
            response = requests.get(f"https://api.spotify.com/v1/audio-features?ids={track_ids_str}", headers=headers)
            if not response.ok:
                retry_after = response.headers.get('Retry-After')
#                 if (retry_after):
#                     print(f"Retrying after {retry_after} seconds")
#                     time.sleep(int(retry_after) + 1)
#                 else:
#                     print(f"Retrying after {retry_after} seconds")
#                     time.sleep(10)
                raise Exception(f'Rate limit Hit for audio_feature retry after : {retry_after}')
            else:
                response_json = response.json()
                return response_json
            
    
    def destructure_artist_data(self,artist_obj):
        return {
            "name": artist_obj.get("name"),
            "id": artist_obj.get("id"),
            "genres": artist_obj.get("genres", []),
            "popularity": artist_obj.get("popularity"),
            "followers": artist_obj.get("followers", {}).get("total")
        }

        
    def get_playlist_by_id(self,playlist_id : str):
        return self.sp.playlist_tracks(playlist_id = playlist_id)    
    
    def get_relevant_artist_data(self, artist_id_list):
            artist_slice = self.artists_df[self.artists_df['id'].isin(artist_id_list)]
            most_popular_artist = artist_slice.loc[artist_slice['popularity'].astype(np.float64).idxmax()]
            combined_popularity = artist_slice['popularity'].astype(np.float64).sum()
            combined_followers = artist_slice['followers'].astype(np.float64).sum()
            most_popular_artist_name = most_popular_artist['name']
            combined_genres_list = artist_slice['genres'].dropna().to_list()
            combined_genres = set([])
            combined_genres.update(*combined_genres_list)
            combined_genres = list(combined_genres)
            
            return {
                "popular_artist": most_popular_artist_name,
                "popular_artist_id": most_popular_artist["id"],
                "combined_genres": list(set(combined_genres)),
                "combined_popularity" : combined_popularity,
                "combined_followers" : combined_followers,
                "artist_popularity": most_popular_artist["popularity"],
                "artist_followers": most_popular_artist["followers"],
            }
    
    def construct_track_and_audio_feat_dict(self,track_json = dict([]), audio_features_json = dict([])):
        track_info = {
            "track_name": track_json.get("name"),
            "track_id": track_json.get("id"),
            "track_number": track_json.get("track_number"),
            "disc_number": track_json.get("disc_number"),
            "duration_ms": track_json.get("duration_ms"),
            "explicit": track_json.get("explicit"),
            "popularity": track_json.get("popularity"),
            "preview_url": track_json.get("preview_url"),
            "isrc": track_json.get("external_ids", {}).get("isrc"),
            "album_name": track_json.get("album", {}).get("name"),
            "album_id": track_json.get("album", {}).get("id"),
            "album_type": track_json.get("album", {}).get("album_type"),
            "album_total_tracks": track_json.get("album", {}).get("total_tracks"),
            "album_release_date": track_json.get("album", {}).get("release_date"),
            "album_release_date_precision": track_json.get("album", {}).get("release_date_precision"),
            "album_images": track_json.get("album", {}).get("images"),
            "artist_names": [artist.get("name") for artist in track_json.get("artists", [])],
            "artist_ids": [artist.get("id") for artist in track_json.get("artists", [])],
            "external_url": track_json.get("external_urls", {}).get("spotify"),
            "acousticness": audio_features_json.get("acousticness"),
            "danceability": audio_features_json.get("danceability"),
            "energy": audio_features_json.get("energy"),
            "instrumentalness": audio_features_json.get("instrumentalness"),
            "key": audio_features_json.get("key"),
            "liveness": audio_features_json.get("liveness"),
            "loudness": audio_features_json.get("loudness"),
            "mode": audio_features_json.get("mode"),
            "speechiness": audio_features_json.get("speechiness"),
            "tempo": audio_features_json.get("tempo"),
            "time_signature": audio_features_json.get("time_signature"),
            "valence": audio_features_json.get("valence"),
        }
        return track_info
    
    def combine_artist_track_audiofeat(self, output_path = ""):
        transformed_series = self.track_audio_feat_df['artist_ids'].apply(self.get_relevant_artist_data).to_list()
        transformed_df = pd.DataFrame(transformed_series)
        assert len(self.track_audio_feat_df) == len(transformed_df)
        combined_df = pd.concat([self.track_audio_feat_df, transformed_df], axis = 1)
        
        SpotifyScraper.df_save_callback(combined_df,output_path,'combined_df.pkl')
        return combined_df
        
    
    @staticmethod
    def df_save_callback(df, output_path, df_name):
        os.makedirs(output_path, exist_ok = True)
        df.to_pickle(os.path.join(output_path,df_name))
        
    @staticmethod
    def _get_multi_threaded_data(max_workers, total_len,func,*args,message = ""):
        data = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # List of tasks to be executed by the thread pool
            tasks = [
                executor.submit(func , *arg) for arg in zip(*args)
            ]
            success = 0
            fails = 0
            
            with tqdm(total=total_len,desc=message) as pbar:
                for future in concurrent.futures.as_completed(tasks):
                    try:
                        data.append(future.result())
                        pbar.update(1)
                        success+=1
                    except Exception as exc:
                        print(f'Task generated an exception: {exc}') 
                        fails+=1
            print(f"success : {success}, fail : {fails}")
                
        return data
    
    def get_several_track_audio_feature(self, track_ids_list, max_workers = 50, output_path = "", df_name = 'track_audio_feat_df.pkl'):
        track_id_set = set(track_ids_list)
        if ('track_id' in self.track_audio_feat_df.columns):
            scraped_track_ids = set(self.track_audio_feat_df['track_id'].astype(str).unique())
            track_id_set.difference_update(scraped_track_ids)
            
        track_ids_list = list(track_id_set)
        for idx in range(0,len(track_ids_list),50):
            track_id_str = ",".join(track_ids_list[idx:idx+50])
            tracks_json = self.get_several_track_info_by_id(track_id_str)["tracks"]
            audio_features_json = self.get_several_audio_feature_by_id(track_ids_str=track_id_str)["audio_features"]
            temp_data = self._get_multi_threaded_data(max_workers,50,self.construct_track_and_audio_feat_dict, tracks_json, audio_features_json, message = f"Done {idx} of {len(track_ids_list)} epoch - {idx//50}")
            self.track_audio_feat_df = pd.concat([self.track_audio_feat_df,pd.DataFrame(temp_data)],ignore_index = True)
            SpotifyScraper.df_save_callback(self.track_audio_feat_df, output_path, df_name)
    
    def get_several_artist_features(self,artist_id_list, max_workers = 50, output_path = "", df_name = 'artists_df.pkl'):
        artist_id_set = set(artist_id_list)
        if ('id' in self.artists_df.columns):
            scraped_artist_ids = set(self.artists_df['id'].astype(str).unique())
            artist_id_set.difference_update(scraped_artist_ids)
            
        artist_id_list = list(artist_id_set)
        for idx in range(0,len(artist_id_list),50):
            artist_id_str = ",".join(artist_id_list[idx:idx+50])
            artists_json = self.get_several_artist_info_by_id(artist_id_str)["artists"]
            temp_data = self._get_multi_threaded_data(max_workers,50,self.destructure_artist_data, artists_json, message = f"Done {idx} of {len(artist_id_list)} epoch - {idx//50}")
            self.artists_df = pd.concat([self.artists_df,pd.DataFrame(temp_data)],ignore_index = True)
            SpotifyScraper.df_save_callback(self.artists_df, output_path, df_name)
    
    def get_video_id_str_from_track_id(self,track_id):
        if ('track_id' not in self.combined_df.columns):
            raise Exception("Please provide a dataframe to get popular_artist and track_id")
        try:
            row = self.combined_df.query(f"`track_id` == '{track_id}'")
            if (row.shape[0] == 0):
                return None
            else:
                row = row.iloc[0]
                track_name = row['track_name']
                artist_name = row['popular_artist']
                search_str = f'{track_name} {artist_name}'
                video_id = self.yt_scraper.get_best_video_id(search_str)
                return video_id
        except Exception as e:
            print(f"error in get_video_id_str_from_track_id message : {e}")
            return None
        
    def get_track_id_video_id_mapping(self,track_ids, max_workers = 50, output_path=  './', df_name = 'track_video_df.pkl'):
        track_video_set = set(track_ids)
        if ('track_id' in self.track_video_df.columns):
            scraped_track_video = set(self.track_video_df['track_id'].astype(str))
            track_video_set.difference_update(scraped_track_video )
        track_ids = list(track_video_set)
        for idx in range(0,len(track_ids),50):
            temp_data = self._get_multi_threaded_data(
                max_workers,
                50,
                self.get_video_id_str_from_track_id,
                track_ids[idx: idx+50],
                message = f"Video_id extraction: {idx}/{len(track_ids)} epoch : {idx//50}/{len(track_ids)//50} ||",
            )
            temp_df = pd.DataFrame(zip(track_ids[idx: idx+50],temp_data), columns = ['track_id', 'video_id'])
            temp_df.dropna(inplace = True)
            self.track_video_df = pd.concat([self.track_video_df,temp_df],ignore_index = True)
            SpotifyScraper.df_save_callback(self.track_video_df, output_path, df_name)
            
        
    
    def scrap(
        self, 
        track_ids,
        output_path = "",
        track_metadata = True, 
        audio_feat_metadata = True,
        audio = True,
        lyrics = True,
    ):
        self.get_several_track_audio_feature(
            track_ids,
            max_workers = 50,
            output_path = output_path,
        )
        unique_artist_ids = set([])
        unique_artist_ids.update(*self.track_audio_feat_df['artist_ids'].dropna().to_list())
        unique_artist_ids = list(unique_artist_ids)
        
        self.get_several_artist_features(
            unique_artist_ids,
            max_workers = 50,
            output_path = output_path,
        )
        
        self.get_track_id_video_id_mapping(
            self.combined_df['track_id'].to_list(), 
            max_workers = 50,
            output_path = output_path,
        )
        
        return self.combine_artist_track_audiofeat(output_path = output_path)
        
    
    
    def get_track_ids_from_playlist(self,*args,**kwargs):
        playlist_data = self.sp.playlist_tracks(*args,**kwargs)
        return [item['track']['id'] for item in playlist_data['items']]
    
    def get_track_ids_from_album(self,*args,**kwargs):
        album_data = self.sp.album_tracks(*args,**kwargs)
        return [item['id'] for item in album_data['items']]
    
    
      

## Loading the Environment and Authentication

### Loading in Kaggle Environment

In [14]:
import os
import json

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

os.environ['KAGGLE_USERNAME'] = secrets.get_secret("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = secrets.get_secret("KAGGLE_KEY")
client_id = secrets.get_secret("SPOTIFY_CLIENT_ID")
client_secret = secrets.get_secret("SPOTIFY_CLIENT_SECRET")
# redirect_uri = secrets.get_secret("REDIRECT_URI")
yt_music_oauth_metadata = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Content-Type": "application/json",
    "X-Goog-AuthUser": "0",
    "x-origin": "https://music.youtube.com",
    "Cookie" : secrets.get_secret("YT_MUSIC_COOKIE")
}

with open(os.path.join('/kaggle/working/','oauth.json'), 'w') as f:
    json.dump(yt_music_oauth_metadata, f)
    
with open('/kaggle/working/tokens.json','w') as f:
    json.dump(eval(secrets.get_secret('YT_TOKEN')), f)

## Scrapper Instance

In [15]:
!cp -r /kaggle/input/spotify-dataset/* /kaggle/working/dataset

In [16]:
def conditionally_load_df(path):
    if (os.path.exists(path)):
        return pd.read_pickle(path)
    else:
        return pd.DataFrame()

In [17]:
scrapper = SpotifyScraper(
    client_id=client_id,
    client_secret=client_secret,
#   redirect_uri=redirect_uri,
    track_audio_feat_df=conditionally_load_df('/kaggle/working/dataset/track_audio_feat_df.pkl'),
    artists_df=conditionally_load_df('/kaggle/working/dataset/artists_df.pkl'),
    combined_df=conditionally_load_df('/kaggle/working/dataset/combined_df.pkl'),
    track_video_df=conditionally_load_df('/kaggle/working/dataset/track_video_df.pkl')
)

In [18]:
with open('/kaggle/input/spotify-metadata-audio-dataset-001/songs.json','r') as f:
    songs_list = json.load(f)

In [19]:
track_ids = songs_list
# track_ids = songs_list[8400:16800] 
# track_ids = songs_list[16800:]

In [20]:
try:
    scrapper.scrap(track_ids, output_path = './dataset')
    !kaggle datasets version -m "Ishan track_video Added" -p /kaggle/working/dataset --dir-mode tar            
except:
#     pass
    !kaggle datasets version -m "Ishan track_video Added" -p /kaggle/working/dataset --dir-mode tar        


Done 0 of 14 epoch - 0:   0%|          | 0/50 [00:00<?, ?it/s]


Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
Task generated an exception: 'NoneType' object has no attribute 'get'
success : 0, fail : 

Video_id extraction: 0/24985 epoch : 0/499 ||: 100%|██████████| 50/50 [00:01<00:00, 43.44it/s]


success : 50, fail : 0


Video_id extraction: 50/24985 epoch : 1/499 ||: 100%|██████████| 50/50 [00:00<00:00, 78.76it/s]


...

success : 50, fail : 0


Video_id extraction: 24900/24985 epoch : 498/499 ||: 100%|██████████| 50/50 [00:04<00:00, 11.81it/s]


success : 50, fail : 0


Video_id extraction: 24950/24985 epoch : 499/499 ||:  70%|███████   | 35/50 [00:04<00:01,  7.82it/s]


success : 35, fail : 0
Starting upload for file combined_df.pkl
100%|██████████████████████████████████████| 19.8M/19.8M [00:00<00:00, 23.5MB/s]
Upload successful: combined_df.pkl (20MB)
Starting upload for file artists_df.pkl
100%|██████████████████████████████████████| 1.43M/1.43M [00:00<00:00, 2.45MB/s]
Upload successful: artists_df.pkl (1MB)
Starting upload for file track_video_df.pkl
100%|████████████████████████████████████████| 951k/951k [00:00<00:00, 1.53MB/s]
Upload successful: track_video_df.pkl (951KB)
Starting upload for file track_audio_feat_df.pkl
100%|██████████████████████████████████████| 17.9M/17.9M [00:00<00:00, 22.2MB/s]
Upload successful: track_audio_feat_df.pkl (18MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ishansrivastava1308/spotify-dataset


In [21]:
# try:
#     track_data = scrapper.get_several_track_data(
#         track_ids[:10],
#         parallelization=0,
#         output_path = 'dataset/songs', 
#         include_audio = False, 
#         include_lyrics = True,
#         max_workers = 1,
#     )
#     track_data_df = pd.DataFrame(track_data)
#     track_data_df.to_pickle('dataset/total_data.pkl')
#     scrapper.df.to_pickle('dataset/extracted_data.pkl')
#     !kaggle datasets version -m "Ishan 0 to 8400 metadata done" -p /kaggle/working/dataset --dir-mode tar
# except Exception as e:
#     print()
#     print(e)
#     print()
#     scrapper.df.to_pickle('dataset/extracted_data.pkl')
#     !kaggle datasets version -m "Ishan 0 to 8400 metadata salvaged" -p /kaggle/working/dataset --dir-mode tar

In [22]:
# try:
#     audio_output_paths  = scrapper.yt_scraper.threaded_download_song_by_video_id_list(track_data_df.video_id.to_numpy(), 'dataset/songs', max_workers = 50)
#     audio_path_df = pd.DataFrame(audio_output_paths)
#     track_data_df.to_pickle('dataset/total_data.pkl')
#     scrapper.df.to_pickle('dataset/extracted_data.pkl')
#     audio_path_df.to_pickle('total_data_audio.pkl')
#     !kaggle datasets version -m "Ishan 0 to 8400 audio done" -p /kaggle/working/dataset --dir-mode tar
# except Exception as e:
#     print()
#     print(e)
#     print()
#     audio_output_paths = scrapper.yt_scraper.threaded_download_song_by_video_id_list(scrapper.df.video_id.to_numpy(), 'dataset/songs', max_workers = 50)
#     audio_path_df = pd.DataFrame(audio_output_paths)
#     audio_path_df.to_pickle('extracted_data_audio.pkl')
#     !kaggle datasets version -m "Ishan 0 to 8400 salavge" -p /kaggle/working/dataset --dir-mode tar    
    
